# Lab 3

## Single Layer Neural Network, Evaluation, and Interpretation

This lab is designed to teach you about different strategies for evaluating neural networks and interpreting the results. For this lab, we are going to study different methods for intrepreting a neural network. We are going to use a modified version of the perceptron learning algorithm. I have modified it to use the sigmoid activation function. This means our learning rule is updated. 

**Note:** Exercises can be autograded and count towards your lab and assignment score. Problems are graded for participation.

In [1]:
from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. change to something else if this is not the case on your system

In [2]:
%load_ext autoreload
%autoreload 2

# make sure your run the cell above before running this
import Lab3_helper

For developing this lab, we can use famous Titanic Kaggle dataset. Description of the data is found https://www.kaggle.com/c/titanic/data.

In [3]:
import pandas as pd
titanic_df = pd.read_csv(
    f"{home}/csc-466-student/data/titanic.csv"
)
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


We need to do some simple preprocessing before our neural network can deal with this data. 

In [4]:
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked']
titanic_df2 = titanic_df.loc[:,features]
titanic_df2['CabinLetter'] = titanic_df2['Cabin'].str.slice(0,1)
X = pd.get_dummies(titanic_df2.drop('Cabin',axis=1)).dropna()
# Standard scaling
means = X.mean()
sds = X.std()
X2 = X.apply(lambda x: (x-means)/sds,axis=1)
X2

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,CabinLetter_A,CabinLetter_B,CabinLetter_C,CabinLetter_D,CabinLetter_E,CabinLetter_F,CabinLetter_G,CabinLetter_T
0,0.910594,-0.530005,0.524203,-0.505541,-0.518614,-0.758520,0.758520,-0.471477,-0.201889,0.537033,-0.130653,-0.259172,-0.277156,-0.212895,-0.20928,-0.125001,-0.075006,-0.037424
1,-1.475329,0.571430,0.524203,-0.505541,0.691412,1.316511,-1.316511,2.118022,-0.201889,-1.859476,-0.130653,-0.259172,3.603025,-0.212895,-0.20928,-0.125001,-0.075006,-0.037424
2,0.910594,-0.254646,-0.551317,-0.505541,-0.505859,1.316511,-1.316511,-0.471477,-0.201889,0.537033,-0.130653,-0.259172,-0.277156,-0.212895,-0.20928,-0.125001,-0.075006,-0.037424
3,-1.475329,0.364911,0.524203,-0.505541,0.347805,1.316511,-1.316511,-0.471477,-0.201889,0.537033,-0.130653,-0.259172,3.603025,-0.212895,-0.20928,-0.125001,-0.075006,-0.037424
4,0.910594,0.364911,-0.551317,-0.505541,-0.503497,-0.758520,0.758520,-0.471477,-0.201889,0.537033,-0.130653,-0.259172,-0.277156,-0.212895,-0.20928,-0.125001,-0.075006,-0.037424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,0.910594,0.640270,-0.551317,5.354136,-0.105246,1.316511,-1.316511,-0.471477,4.946280,-1.859476,-0.130653,-0.259172,-0.277156,-0.212895,-0.20928,-0.125001,-0.075006,-0.037424
886,-0.282368,-0.185807,-0.551317,-0.505541,-0.409958,-0.758520,0.758520,-0.471477,-0.201889,0.537033,-0.130653,-0.259172,-0.277156,-0.212895,-0.20928,-0.125001,-0.075006,-0.037424
887,-1.475329,-0.736524,-0.551317,-0.505541,-0.088711,1.316511,-1.316511,-0.471477,-0.201889,0.537033,-0.130653,3.853031,-0.277156,-0.212895,-0.20928,-0.125001,-0.075006,-0.037424
889,-1.475329,-0.254646,-0.551317,-0.505541,-0.088711,-0.758520,0.758520,2.118022,-0.201889,-1.859476,-0.130653,-0.259172,3.603025,-0.212895,-0.20928,-0.125001,-0.075006,-0.037424


In [5]:
t = titanic_df.loc[X2.index,'Survived']
t

0      0
1      1
2      1
3      1
4      0
      ..
885    0
886    0
887    1
889    1
890    0
Name: Survived, Length: 714, dtype: int64

#### Problem 1
In your own words, describe the preprocessing steps I took above.

Your solution here.

### Training the network
We are now going to train the network. We'll use the defaults that I've set in the function, but feel free to change them around and see how you can do. I am going to show you how setting the seed makes a difference in training the algorithm.

In [6]:
seeds = [0,1,2,3,4,5]
results = None
w = {}
X_test = {}
t_test = {}
for seed in seeds:
    w[seed],X_test[seed],t_test[seed],results1 = Lab3_helper.train(X2,t,seed=seed)
    if results is None:
        results = results1
    else:
        results = results.append(results1)

In [7]:
import altair as alt
alt.data_transformers.disable_max_rows()

source = results.reset_index().drop(['n','test_size','val_size'],axis=1).melt(id_vars=['epoch','seed'])

alt.Chart(source).mark_line().encode(
    x='epoch',
    y=alt.Y('value',title='Accuracy'),
    color='variable',
    column='seed'
)

alt.Chart(...)

#### Problem 2
Run a similar experiment but vary the learning rate as below. Keep the seed constant (seed=0). What do the graphs tell you about the parameter ``n`` (i.e., $\eta$)?

In [8]:
ns = [0.1,0.2,0.5,0.75,1]
results = None
# Your solution here

In [9]:
source = results.reset_index().drop(['seed','test_size','val_size'],axis=1).melt(id_vars=['epoch','n'])

alt.Chart(source).mark_line().encode(
    x='epoch',
    y=alt.Y('value',title='Accuracy'),
    color='variable',
    column='n'
)

alt.Chart(...)

#### Exercise 1
The first step to evaluating any classification problem is establishing a baseline. Write a function that calculates the baseline accuracy if you predict the majority class on the test dataset.

In [10]:
from sklearn.model_selection import train_test_split
import numpy as np

seeds = [0,1,2,3,4,5]
results = pd.DataFrame(columns=["seed","frac_max_class","accuracy_test","accuracy_train2","accuracy_val"]).set_index("seed")
for seed in seeds:
    np.random.seed(seed)
    X_train, X_test, t_train, t_test = train_test_split(X2, t, test_size=0.3)
    X_train2, X_val, t_train2, t_val = train_test_split(X_train, t_train, test_size=0.3)

    ## Your solution in evaluate_baseline(...)
    frac_max_class,accuracy_test,accuracy_train2,accuracy_val = Lab3_helper.evaluate_baseline(t_test,t_train2,t_val)
    
    results = results.append(pd.Series([frac_max_class,accuracy_test,accuracy_train2,accuracy_val],index=results.columns,name=seed))

results

,frac_max_class,accuracy_test,accuracy_train2,accuracy_val
seed,,,,
0,0.593123,0.581395,0.593123,0.613333
1,0.538682,0.623256,0.538682,0.680000
2,0.573066,0.623256,0.573066,0.600000
3,0.581662,0.604651,0.581662,0.606667
4,0.598854,0.576744,0.598854,0.606667
5,0.558739,0.604651,0.558739,0.660000


In [11]:
import altair as alt

source = results.melt()

alt.Chart(source).mark_boxplot().encode(
    x='variable:N',
    y=alt.Y('value:Q',scale=alt.Scale(domain=(0.5, 0.7)))
)

alt.Chart(...)

#### Exercise 2
Write a function that makes predictions for an X matrix using the weights.

In [12]:
w,X_test,t_test,results = Lab3_helper.train(X2,t,seed=0)

y_test = Lab3_helper.predict(w,X_test)
y_test

423    0
177    1
305    1
292    0
889    1
      ..
203    0
499    0
628    0
879    1
745    0
Length: 215, dtype: int64

#### Exercise 3
Write a function that calculates the confusion matrix.

In [13]:
y_test = Lab3_helper.predict(w,X_test)

cm = Lab3_helper.confusion_matrix(t_test,y_test,labels=[0,1])
cm

,0,1
0,98,27
1,18,72


#### Exercise 4
Sensitivity, recall, hit rate, or true positive rate (TPR)

${\displaystyle \mathrm {TPR} ={\frac {\mathrm {TP} }{\mathrm {P} }}={\frac {\mathrm {TP} }{\mathrm {TP} +\mathrm {FN} }}=1-\mathrm {FNR} }$

specificity, selectivity or true negative rate (TNR)

${\displaystyle \mathrm {TNR} ={\frac {\mathrm {TN} }{\mathrm {N} }}={\frac {\mathrm {TN} }{\mathrm {TN} +\mathrm {FP} }}=1-\mathrm {FPR} }$

precision or positive predictive value (PPV)

${\displaystyle \mathrm {PPV} ={\frac {\mathrm {TP} }{\mathrm {TP} +\mathrm {FP} }}=1-\mathrm {FDR} }$

F1

${\displaystyle F_{1}={\frac {2}{\mathrm {recall^{-1}} +\mathrm {precision^{-1}} }}=2\cdot {\frac {\mathrm {precision} \cdot \mathrm {recall} }{\mathrm {precision} +\mathrm {recall} }}={\frac {\mathrm {tp} }{\mathrm {tp} +{\frac {1}{2}}(\mathrm {fp} +\mathrm {fn} )}}}$

Write a function that calculates accuracy, sensitivity/recall, specificity, precision, and F1

In [14]:
stats = Lab3_helper.evaluation(cm,positive_class=1)
stats

{'accuracy': 0.7906976744186046,
 'sensitivity/recall': 0.8,
 'specificity': 0.784,
 'precision': 0.7272727272727273,
 'F1': 0.761904761904762}

In [15]:
stats = Lab3_helper.evaluation(cm,positive_class=0)
stats

{'accuracy': 0.7906976744186046,
 'sensitivity/recall': 0.784,
 'specificity': 0.8,
 'precision': 0.8448275862068966,
 'F1': 0.8132780082987552}

#### Exercise 5
Create a function that trains our neural network for each of the seeds and then returns variable importance of each feature as:

${\it importance}(w_i) = \frac{1}{|seeds|}\sum_{s \in seeds} \frac{\sqrt{w_i^2}}{max\left(\sqrt{w_0^2}, \sqrt{w_1^2} ... \sqrt{w_d^2}\right)}$

Basically, compute the variable importance for each seed and then average.

In [16]:
seeds = [0,1,2,3,4,5]
importances = Lab3_helper.importance(X2,t,seeds)
importances.sort_values(ascending=False)

Age              0.774353
Sex_female       0.685446
Sex_male         0.674874
Pclass           0.521585
CabinLetter_D    0.304603
Parch            0.254741
CabinLetter_A    0.240389
CabinLetter_G    0.235199
CabinLetter_E    0.223833
CabinLetter_F    0.218563
Fare             0.195130
SibSp            0.194752
Embarked_Q       0.170747
CabinLetter_B    0.170171
CabinLetter_T    0.156553
Embarked_S       0.156516
Embarked_C       0.147504
CabinLetter_C    0.124295
dtype: float64

#### Problem 3: Compare these variable importances to the variable importances achieved by test-based permutation and train-based variable importances. 

To complete this problem, you will have to copy your previous lab's solutions to this notebook or Lab3_helper.py. From there, you can make the modifications necessary to train and evaluate the neural network instead of the Bayesian classifier.

In [17]:
# Your solution here

**Your interpretation here**

In [18]:
# Good job!
# Don't forget to push with ./submit.sh

#### Having trouble with the test cases and the autograder?

You can always load up the answers for the autograder. The autograder runs your code and compares your answer to the expected answer. I manually review your code, so there is no need to hide this from you.

In [2]:
import joblib
answers = joblib.load(f"{home}/csc-466-student/tests/answers_Lab3.joblib")
answers.keys()

dict_keys(['exercise_1', 'exercise_2', 'exercise_3', 'exercise_4', 'exercise_5'])

In [4]:
answers['exercise_2']

423    0
177    1
305    1
292    0
889    1
      ..
203    0
499    0
628    0
879    1
745    0
Length: 215, dtype: int64